## Imports

In [ ]:
import os

from mmaction.datasets import build_dataset, build_dataloader
from mmaction.models import build_model
from mmcv import Config

## Loading batches

In [ ]:
cfg = Config.fromfile('./baseline_v2.py')

In [ ]:
os.chdir('../../..')

In [ ]:
cfg.data.train

In [ ]:
train_dataset = build_dataset(cfg=cfg.data.train)
train_loader = build_dataloader(
        train_dataset,
        videos_per_gpu=5,
        workers_per_gpu=4,
        persistent_workers=False,
        num_gpus=1,
        dist=False)

val_dataset = build_dataset(cfg=cfg.data.val)
val_loader = build_dataloader(
        val_dataset,
        videos_per_gpu=1,
        workers_per_gpu=4,
        persistent_workers=False,
        num_gpus=1,
        dist=False)

## Learning Hyperparameters

In [ ]:
import optuna
import torch.nn as nn
import torch.optim as optim
from mmcv import Config
import torch
import os
import logging

# Configure logging
logging.basicConfig(filename='optuna_training_adam.log', 
                    filemode='w', 
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    level=logging.INFO)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Database file path for saving study
db_file = "sqlite:///optuna_study_adam.db"

sampler = optuna.samplers.TPESampler(seed=42)

# Set up study with the option to resume if it already exists
study = optuna.create_study(
    sampler=sampler,
    direction="maximize", 
    study_name="my_study", 
    storage=db_file,
    load_if_exists=True
)

def objective(trial):
    # Hyperparameters to tune
    dropout_ratio = trial.suggest_float("dropout_ratio", 0.3, 0.7)
    lr = trial.suggest_loguniform("lr", 1e-8, 1e-5)
    max_norm = trial.suggest_int("max_norm", 1, 50)
    
    # Backbone parameters
    cfg.model.backbone.with_pool2 = trial.suggest_categorical("with_pool2", [True, False])
    cfg.model.backbone.bottleneck_mode = trial.suggest_categorical("bottleneck_mode", ["ir", "ip"])
    cfg.model.backbone.norm_eval = trial.suggest_categorical("norm_eval", [True, False])
    cfg.model.backbone.bn_frozen = trial.suggest_categorical("bn_frozen", [True, False])
    
    # Fixed pretrained URL
    cfg.model.backbone.pretrained = 'https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth'

    # Adjust config parameters
    cfg.model.cls_head.dropout_ratio = dropout_ratio
    
    # Initialize model, criterion, optimizer, scheduler
    model = build_model(cfg.model, train_cfg=None, test_cfg=cfg.get('test_cfg')).to(device)
    
    # Use Adam optimizer
    optimizer = optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=0.00001
    )
    
    # Early stopping parameters
    total_epochs = 40
    eval_interval = 1
    patience = 5
    best_val_accuracy = 0.0
    epochs_without_improvement = 0

    for epoch in range(total_epochs):

        # Training loop
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        
        for _, data in enumerate(train_loader):
            inputs, labels = data['imgs'].to(device), data['label'].to(device)
            
            optimizer.zero_grad()
            results = model(inputs, labels, return_loss=True)
            loss = results['loss_cls']
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            running_loss += loss.item()
            correct += (results['top1_acc'] * inputs.size(0))
            total += inputs.size(0)

        train_accuracy = correct / total
        train_loss = running_loss / len(train_loader)

        logging.info(f"Epoch [{epoch + 1}/{total_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

        # Validation loop (every `eval_interval` epochs)
        if (epoch + 1) % eval_interval == 0:
            model.eval()
            val_running_loss, val_correct, val_total = 0.0, 0, 0
            with torch.no_grad():
                for val_data in val_loader:
                    val_inputs, val_labels = val_data['imgs'].to(device), val_data['label'].to(device)
                    
                    val_results = model(val_inputs, val_labels, return_loss=True)
                    val_loss = val_results['loss_cls']
                    val_running_loss += val_loss.item()
                    val_correct += (val_results['top1_acc'] * val_inputs.size(0))
                    val_total += val_inputs.size(0)

            val_accuracy = val_correct / val_total
            val_loss = val_running_loss / len(val_loader)

            # Report validation accuracy to Optuna
            trial.report(val_accuracy, epoch)

            # Check if validation accuracy improved
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                epochs_without_improvement = 0  # Reset counter
            else:
                epochs_without_improvement += 1

            # Early stopping check
            if epochs_without_improvement >= patience:
                logging.info(f"Early stopping at epoch {epoch + 1} due to no improvement in validation accuracy.")
                break

            # Prune unpromising trials
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
    
    return best_val_accuracy


# Run Optuna Study
study.optimize(objective, n_trials=50)

logging.info("Best hyperparameters: %s", study.best_params)
logging.info("Best validation accuracy: %f", study.best_value)

## Results

In [ ]:
# Retrieve all trials and print their parameters
for trial in study.trials:
    print(f"Trial number: {trial.number}")
    print(f"Parameters: {trial.params}")
    print(f"Value (e.g., validation accuracy): {trial.value}")
    print("-" * 30)

In [ ]:
best_trial = study.best_trial
print("Best trial number:", best_trial.number)
print("Best parameters:", best_trial.params)
print("Best validation accuracy:", best_trial.value)